# Notebook 2: Classical IR Models and Evaluation

This notebook covers **Steps 2 and 3** of the project. We will:

1.  **Load Processed Data**: Import the cleaned corpus and queries from Notebook 1.
2.  **Implement TF-IDF**: Build a TF-IDF Vector Space Model from scratch.
3.  **Implement BM25**: Use the `rank_bm25` library for the Okapi BM25 model.
4.  **Rank Documents**: Retrieve and rank documents for a sample of 10 queries using both models.
5.  **Evaluate Performance**: Calculate Precision@k, Recall@k, MAP, MRR, and nDCG for both models and compare their performance.

## 1. Setup and Installation

We need `numpy` and `scipy` for mathematical operations (especially for TF-IDF), and `rank_bm25` for the BM25 model.

In [1]:
!pip install numpy scipy rank_bm25 pandas

Defaulting to user installation because normal site-packages is not writeable


## 2. Load Processed Data

Load the dataframes and the inverted index that we saved in the previous notebook.

In [2]:
import pandas as pd
import json
import os
import numpy as np
from tqdm import tqdm

DATA_DIR = '../fiqa/processed_data'

# Load data
corpus_df = pd.read_pickle(os.path.join(DATA_DIR, 'corpus_processed.pkl'))
queries_df = pd.read_pickle(os.path.join(DATA_DIR, 'queries_processed.pkl'))
qrels_df = pd.read_pickle(os.path.join(DATA_DIR, 'qrels.pkl'))
with open(os.path.join(DATA_DIR, 'inverted_index.json'), 'r', encoding='utf-8') as f:
    inverted_index = json.load(f)

# Create mappings for faster lookups
query_id_to_text = pd.Series(queries_df.processed_text.values, index=queries_df.query_id).to_dict()

print("Data loaded successfully.")
display(corpus_df.head())

Data loaded successfully.


,doc_id,text,processed_text
0,3,I'm not saying I don't like the idea of on-the...,"[say, like, idea, training, ca, expect, compan..."
1,31,So nothing preventing false ratings besides ad...,"[nothing, prevent, false, rating, besides, add..."
2,56,You can never use a health FSA for individual ...,"[never, use, health, fsa, individual, health, ..."
3,59,Samsung created the LCD and other flat screen ...,"[samsung, create, lcd, flat, screen, technolog..."
4,63,Here are the SEC requirements: The federal sec...,"[sec, requirement, federal, security, law, def..."


## 3. TF-IDF Vector Space Model Implementation

We'll build the components for TF-IDF step-by-step:
- **TF (Term Frequency)**: How often a term appears in a document.
- **IDF (Inverse Document Frequency)**: A measure of how important a term is across the whole corpus.
- **TF-IDF Weight**: The product of TF and IDF.
- **Ranking**: Use cosine similarity between a query vector and document vectors to find the most similar documents.

In [3]:
from collections import Counter
import math
from scipy.spatial.distance import cosine

class TFIDFModel:
    def __init__(self, corpus_df, inverted_index):
        self.corpus = corpus_df
        self.inverted_index = inverted_index
        self.doc_lengths = {row['doc_id']: len(row['processed_text']) for _, row in self.corpus.iterrows()}
        self.doc_term_freqs = {row['doc_id']: Counter(row['processed_text']) for _, row in self.corpus.iterrows()}
        self.num_docs = len(self.corpus)
        self.vocab = list(self.inverted_index.keys())
        self.idf = self._calculate_idf()
    
    def _calculate_idf(self):
        idf = {}
        for term in tqdm(self.vocab, desc="Calculating IDF"):
            doc_freq = len(self.inverted_index.get(term, []))
            idf[term] = math.log(self.num_docs / (1 + doc_freq)) # Add 1 to avoid division by zero
        return idf

    def _get_doc_vector(self, doc_id):
        term_freqs = self.doc_term_freqs[doc_id]
        vector = np.zeros(len(self.vocab))
        for i, term in enumerate(self.vocab):
            tf = term_freqs.get(term, 0) / self.doc_lengths[doc_id]
            vector[i] = tf * self.idf.get(term, 0)
        return vector
        
    def _get_query_vector(self, query_terms):
        query_term_freqs = Counter(query_terms)
        vector = np.zeros(len(self.vocab))
        for i, term in enumerate(self.vocab):
            tf = query_term_freqs.get(term, 0) / len(query_terms)
            vector[i] = tf * self.idf.get(term, 0)
        return vector

    def rank_documents(self, query_terms, top_n=100):
        query_vec = self._get_query_vector(query_terms)
        scores = {}
        
        # Candidate selection: only score docs that contain at least one query term
        candidate_docs = set()
        for term in query_terms:
            candidate_docs.update(self.inverted_index.get(term, []))

        if not candidate_docs:
            return []
            
        for doc_id in tqdm(list(candidate_docs), desc="Ranking Docs (TF-IDF)", leave=False):
            doc_vec = self._get_doc_vector(doc_id)
            # Cosine similarity is 1 - cosine distance
            score = 1 - cosine(query_vec, doc_vec)
            if not np.isnan(score):
                scores[doc_id] = score
        
        ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        return ranked_docs[:top_n]

# Initialize the model (this will take a moment to calculate IDF)
tfidf_model = TFIDFModel(corpus_df, inverted_index)

Calculating IDF: 100%|██████████| 53666/53666 [00:00<00:00, 1745978.27it/s]


## 4. BM25 Model Implementation

For BM25, we'll use the efficient `rank_bm25` library. It's a highly optimized implementation of the Okapi BM25 ranking function.

In [4]:
from rank_bm25 import BM25Okapi

# BM25 requires a list of tokenized documents
tokenized_corpus = corpus_df['processed_text'].tolist()
doc_ids = corpus_df['doc_id'].tolist()

bm25 = BM25Okapi(tokenized_corpus, k1=1.2, b=0.6)

# Create a mapping from index to doc_id for retrieving results
idx_to_doc_id = {i: doc_id for i, doc_id in enumerate(doc_ids)}

def rank_documents_bm25(query_terms, top_n=100):
    doc_scores = bm25.get_scores(query_terms)
    top_indices = np.argsort(doc_scores)[::-1][:top_n]
    
    results = []
    for idx in top_indices:
        doc_id = idx_to_doc_id[idx]
        score = doc_scores[idx]
        results.append((doc_id, score))
    return results

## 5. Run Retrieval for Sample Queries

Let's select 10 queries from our dataset and run both retrieval models to get ranked lists of documents.

In [ ]:

rel_counts = qrels_df.groupby('query_id').size()

rich_queries = rel_counts[rel_counts > 5]


sample_query_ids = rich_queries.index[:10].tolist()

results_tfidf = {}
results_bm25 = {}

for qid in tqdm(sample_query_ids, desc="Running Queries"):
    query_terms = query_id_to_text[qid]
    
    # Get TF-IDF results
    tfidf_ranked_list = tfidf_model.rank_documents(query_terms, top_n=100)
    results_tfidf[qid] = [doc_id for doc_id, score in tfidf_ranked_list]
    
    # Get BM25 results
    bm25_ranked_list = rank_documents_bm25(query_terms, top_n=100)
    results_bm25[qid] = [doc_id for doc_id, score in bm25_ranked_list]

print("\n--- Sample BM25 Results for Query ID:", sample_query_ids[0])
print(results_bm25[sample_query_ids[0]][:5])

Running Queries: 100%|██████████| 10/10 [30:57<00:00, 185.74s/it] 


--- Sample BM25 Results for Query ID: 10028
['25466', '163216', '112435', '413169', '556072']


## 6. Evaluation

Now we implement the evaluation metrics and apply them to the results from both models.

First, let's structure our relevance data (qrels) into a more usable format.

In [8]:
# Create a dictionary for easy lookup of relevant documents for each query
relevant_docs = qrels_df.groupby('query_id')['doc_id'].apply(list).to_dict()

# --- Metric Implementations ---

def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    return len(set(retrieved_k) & set(relevant)) / k

def recall_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    if not relevant:
        return 0.0
    return len(set(retrieved_k) & set(relevant)) / len(relevant)

def average_precision(retrieved, relevant):
    if not relevant:
        return 0.0
    
    hits = 0
    sum_precisions = 0.0
    for i, doc_id in enumerate(retrieved):
        if doc_id in relevant:
            hits += 1
            sum_precisions += hits / (i + 1)
    
    return sum_precisions / len(relevant)

def mean_average_precision(results, relevant_docs):
    aps = [average_precision(results[qid], relevant_docs.get(qid, [])) for qid in results]
    return np.mean(aps)

def mean_reciprocal_rank(results, relevant_docs):
    rrs = []
    for qid in results:
        relevant = relevant_docs.get(qid, [])
        for i, doc_id in enumerate(results[qid]):
            if doc_id in relevant:
                rrs.append(1 / (i + 1))
                break
        else: # If no relevant doc was found
            rrs.append(0.0)
    return np.mean(rrs)

def ndcg_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    dcg = 0.0
    for i, doc_id in enumerate(retrieved_k):
        if doc_id in relevant:
            dcg += 1 / math.log2(i + 2) # relevance=1
    
    idcg = 0.0
    num_relevant_k = min(len(relevant), k)
    for i in range(num_relevant_k):
        idcg += 1 / math.log2(i + 2)
        
    return dcg / idcg if idcg > 0 else 0.0

def evaluate_model(results, relevant_docs, k_values=[1,3,5, 10]):
    metrics = {}
    for k in k_values:
        precisions = [precision_at_k(results[qid], relevant_docs.get(qid, []), k) for qid in results]
        recalls = [recall_at_k(results[qid], relevant_docs.get(qid, []), k) for qid in results]
        ndcgs = [ndcg_at_k(results[qid], relevant_docs.get(qid, []), k) for qid in results]
        metrics[f'P@{k}'] = np.mean(precisions)
        metrics[f'R@{k}'] = np.mean(recalls)
        metrics[f'nDCG@{k}'] = np.mean(ndcgs)
    
    metrics['MAP'] = mean_average_precision(results, relevant_docs)
    metrics['MRR'] = mean_reciprocal_rank(results, relevant_docs)
    
    return metrics

## 7. Compare Model Performance

Finally, let's compute the metrics for both TF-IDF and BM25 and display them in a table for easy comparison.

In [ ]:


tfidf_metrics = evaluate_model(results_tfidf, relevant_docs)
bm25_metrics = evaluate_model(results_bm25, relevant_docs)

comparison_df = pd.DataFrame([tfidf_metrics, bm25_metrics], index=['TF-IDF', 'BM25'])

print("--- Model Performance Comparison ---")
display(comparison_df)

{'10028': ['112435', '510373', '565698', '455428', '88774', '476068', '534975', '235624', '403314', '494553', '92038', '212320', '91926', '36086', '413169', '489539', '323475', '418235', '364099', '184339', '468831', '341837', '150607', '243520', '58353', '251642', '386994', '510805', '221364', '473865', '288504', '423403', '275444', '523058', '189881', '502333', '58433', '556072', '237197', '219536', '161608', '360139', '560633', '163216', '84630', '47054', '244254', '381610', '233795', '390642', '387722', '462183', '109903', '559370', '303360', '187739', '27268', '364802', '439593', '459423', '98294', '390474', '529954', '494148', '234286', '388160', '127165', '284153', '105707', '187201', '219061', '233294', '587624', '466778', '488638', '432961', '102088', '432307', '25466', '310120', '542463', '109227', '30808', '340209', '85488', '263867', '295089', '213045', '212673', '445639', '107152', '310629', '83543', '560915', '397455', '273179', '482963', '274108', '404605', '560340'], '1

,P@1,R@1,nDCG@1,P@3,R@3,nDCG@3,P@5,R@5,nDCG@5,P@10,R@10,nDCG@10,MAP,MRR
TF-IDF,0.1,0.014286,0.1,0.066667,0.028571,0.076536,0.08,0.057143,0.084528,0.07,0.102381,0.097636,0.064430,0.194162
BM25,0.2,0.030952,0.2,0.133333,0.061905,0.140784,0.14,0.107143,0.142596,0.09,0.135714,0.139263,0.083314,0.287730


### Analysis

When you run the evaluation, you will likely observe that **BM25 consistently outperforms TF-IDF** across most metrics. Here's why:

- **Document Length Normalization**: BM25 has a more sophisticated way of penalizing long documents. TF-IDF's cosine normalization can sometimes unfairly favor shorter documents. BM25's `b` parameter controls this normalization more effectively.
- **Term Frequency Saturation**: BM25's `k1` parameter prevents terms that appear very frequently in a document from having an overly dominant effect on the score. In TF-IDF, the term frequency component grows linearly and is not capped, which can sometimes be a problem.

Overall, BM25 is a probabilistic model that is generally considered the state-of-the-art for classical keyword-based information retrieval and is a much stronger baseline than the standard TF-IDF Vector Space Model.

In [ ]:
rel_counts = qrels_df.groupby('query_id')['doc_id'].nunique()
print(rel_counts.loc[sample_query_ids])